In [6]:
from src.preprocess import get_train_data_from_sim_data

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
csv_file = '../sim_data_2/driving_log.csv'
image_file = '../sim_data_2/IMG/'
X_train, y_train = get_train_data_from_sim_data(csv_file, image_file)

In [9]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3))) # preprocessing  160x320x3
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Conv2D(filters=6, kernel_size=[5,5], strides=[1,1], padding='same', activation='relu'))  #160x320x6
model.add(MaxPooling2D(pool_size=[2,2], strides=None, padding='same')) # 80 x160x6
model.add(Conv2D(filters=16, kernel_size=[5,5], strides=[1,1], padding='same', activation='relu')) #80 x160x16
model.add(MaxPooling2D(pool_size=[2,2], strides=None, padding='same')) # 40x 80 x16 
model.add(Conv2D(filters=32, kernel_size=[5,5], strides=[1,1], padding='same', activation='relu')) #40x 80x32
model.add(MaxPooling2D(pool_size=[2,2], strides=None, padding='same')) # 20x 40 x32
model.add(Conv2D(filters=64, kernel_size=[5,5], strides=[1,1], padding='same', activation='relu')) #20x 40x64
model.add(MaxPooling2D(pool_size=[2,2], strides=None, padding='same')) # 10x 20 x64
model.add(Flatten()) # 12800
model.add(Dense(2480, activation='relu'))
model.add(Dense(516, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train,batch_size=16 ,validation_split=0.2, shuffle=True, nb_epoch=7)
model.save('model.h5')

D:\Anaconda3\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 7665 samples, validate on 1917 samples
Epoch 1/7
7665/7665 [==============================] - 61s - loss: 0.1191 - val_loss: 0.1109
Epoch 2/7
7665/7665 [==============================] - 52s - loss: 0.0838 - val_loss: 0.1092
Epoch 3/7
7665/7665 [==============================] - 53s - loss: 0.0754 - val_loss: 0.1282
Epoch 4/7
7665/7665 [==============================] - 52s - loss: 0.0663 - val_loss: 0.1348
Epoch 5/7
7665/7665 [==============================] - 53s - loss: 0.0571 - val_loss: 0.1485
Epoch 6/7
5984/7665 [======================>.......] - ETA: 11s - loss: 0.0480